# Calculate Vacancy Formation Energy in Silicon using MatterSim Potential

This notebook demonstrates how to compute the vacancy formation energy in silicon using the MatterSim potential within the Mat3ra framework.

## Methodology

The vacancy formation energy is calculated using the following approach:

1. **Create pristine supercell**: Build a silicon supercell from the unit cell and relax it using MatterSim potential
2. **Create vacancy**: Remove a single atom from the center of the supercell
3. **Relax vacancy structure**: Optimize the geometry of the defective supercell
4. **Calculate formation energy**: Compute the energy difference according to the formula:
   
   E_formation = E_vacancy - (N_vacancy/N_bulk) × E_bulk
   
   where:
   - E_vacancy: Total energy of the supercell with vacancy
   - E_bulk: Total energy of the pristine supercell
   - N_vacancy: Number of atoms in vacancy supercell
   - N_bulk: Number of atoms in pristine supercell

In [56]:
# Install required packages if not already installed
# INFO: if not installed correctly, clear environment and run `pip install .[forcefields]` in the terminal
try:
    import mattersim
except ImportError:
    import subprocess, sys

    subprocess.run([sys.executable, "-m", "pip", "install", ".[forcefields]", "--quiet"], check=False)
    import mattersim

## 1. Prepare materials
### 1.1. Load material data

In [57]:
from ase.io import read
from mat3ra.made.material import Material
from mat3ra.made.tools.convert import from_ase

FOLDER = "data"
FILE_NAME = "generated_crystals.extxyz"
atoms = read(f"{FOLDER}/{FILE_NAME}")

original_material = Material.create(from_ase(atoms))

### 1.2. Add modifications to material
Use any of the tools to modify the material, e.g. create supercell, add vacancy, add interstitials, etc.
Below is the example of creating a supercell and adding a vacancy.

In [58]:
# from mat3ra.made.tools.helpers import create_vacancy, create_supercell

# supercell = create_supercell(original_material, scaling_factor=[2,2,2])
# material = create_vacancy(supercell, coordinate=[0.5, 0.5, 0.5], placement_method="closest_site")

## 1.3. Visualize materials

In [59]:
from utils.visualize import visualize_materials

visualize_materials([material], rotation="-90x,-90y")

GridBox(children=(VBox(children=(Label(value='O6Si2 - Material - rotation: -90x,-90y', layout=Layout(align_sel…

## 2. Setup calculation
### 2.1. Convert to ASE atoms

In [60]:
from mat3ra.made.tools.convert import to_ase

original_material_atoms = to_ase(original_material)
material_atoms = to_ase(material)

### 2.2. Setup MatterSim calculator

In [61]:
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer

original_material_atoms.calc = MatterSimCalculator()
material_atoms.calc = MatterSimCalculator()

### 2.3. Relax structures

In [62]:
relaxer = Relaxer(optimizer="BFGS", constrain_symmetry=True)
relaxer.relax(material_atoms, steps=500)  # In-place relaxation


      Step     Time          Energy          fmax
BFGS:    0 18:31:51      -51.711601        7.474610
BFGS:    1 18:31:51      -52.868916        3.378879
BFGS:    2 18:31:51      -53.223766        0.925829
BFGS:    3 18:31:51      -53.253685        0.452392
BFGS:    4 18:31:51      -53.267704        0.363445
BFGS:    5 18:31:51      -53.279449        0.310323
BFGS:    6 18:31:51      -53.286488        0.260276
BFGS:    7 18:31:51      -53.290512        0.309296
BFGS:    8 18:31:51      -53.295967        0.394289
BFGS:    9 18:31:51      -53.305187        1.008522
BFGS:   10 18:31:51      -53.322845        0.716170
BFGS:   11 18:31:51      -53.378014        0.674018
BFGS:   12 18:31:51      -53.508362        2.157271
BFGS:   13 18:31:51      -53.562679        0.714065
BFGS:   14 18:31:51      -53.587223        0.651299
BFGS:   15 18:31:51      -53.609467        0.720237
BFGS:   16 18:31:51      -53.688854        0.940445
BFGS:   17 18:31:51      -53.783585        1.254217
BFGS:   18 18:

(True,
 MSONAtoms(symbols='SiOSiO5', pbc=True, cell=[[4.810662746141784, 1.3666731481439e-15, 2.0036156176167834], [2.0573911374447738, 4.463548981818283, 1.628575563353106], [1.8110774840280817e-16, 2.1009441273818866e-16, 6.223104953999998]], calculator=MatterSimCalculator(...)))

### 2.4. Visualize relaxed structures

In [63]:
from mat3ra.made.tools.convert import from_ase

relaxed_material = Material.create(from_ase(material_atoms))

visualize_materials([original_material, relaxed_material], rotation="-90x,-90y")

GridBox(children=(VBox(children=(Label(value='O6Si2 - Material - rotation: -90x,-90y', layout=Layout(align_sel…

## 3. Calculate energy differences

### 3.1. Get energies and atom counts

In [64]:
energy_relaxed = material_atoms.get_potential_energy()
energy_original = original_material_atoms.get_potential_energy()

### 3.2. Calculate energy difference

In [65]:
e_difference = energy_relaxed - energy_original
print(f"Energy difference: {e_difference} eV")

n_original = len(original_material_atoms)
n_relaxed = len(material_atoms)

energy_original_per_atom = energy_original / n_original
energy_difference_per_atom = e_difference / n_relaxed

print(f"Energy difference per atom: {energy_difference_per_atom} eV/atom")


Energy difference: -3.109821319580078 eV
Energy difference per atom: -0.38872766494750977 eV/atom
